  # AI on the edge

Deep learning on small devices


- Motivation
- Deep learning models
- Running on devices

## Why AI on devices?

![cloud](assets/lsvm-machine-learning-cloud.jpg)

![data center](assets/how_safe_datacenter.jpg)

<img src="assets/GDP-Complience-V02-72dpi-RGB-01.jpg" width="70%">

<img src="assets/qualcomm_on_device_intelligence.png" width="80%">

## Deep learning models for devices

## MobileNet

https://arxiv.org/abs/1704.04861v1

Standard convolution $h_i.w_i.d_i.d_j.k.k$

Depthwise separable $h_i.w_i.d_i(d_j+k^2)$

<img src="assets/depthwise_separable_conv_unit.png" width="400" style="float:left;">
<img src="assets/depthwise_separable_conv_layer.png" width="400" style="float:left;">

<img src="assets/mobilenets.png" width="500">

<img src="assets/mobilenet_res.png" width=900>

<img src="assets/mobilenet_acc.png" width=40%>

## MobileNet V2

https://arxiv.org/abs/1801.04381

<img src="assets/mobilenetv2_unit.png" width=60%>

<img src="assets/mobilenetv2_block.png" width=450 style="float:left;">
<img src="assets/mobilenetv2_arch.png" width=450 style="float:left;">

<img src="assets/mobilenetv2_curve.png">

<img src="assets/mobilenetv2_compare.png" width=500>

<img src="assets/mobilenetv2_residual.png" width=70%>

<img src="assets/mobilenetv2_residual_curve.png" width=70%>

## SqueezeNet

https://arxiv.org/abs/1602.07360

https://github.com/DeepScale/SqueezeNet
<img src="assets/squeezenet_unit.png" width=800>

<img src="assets/squeezenet_arch.png" width=750>
<img src="assets/squeezenet_bypass.png" width=750>

<img src="assets/squeezenet_compare.png" width=800>

## Running models on devices

## CoreML

https://developer.apple.com/documentation/coreml

- download pre-trained .mlmodel
  - or convert from Caffe, Tensorflow, Onyx, etc
- add to XCode project to generate wrapper
  - use optional Vision library to preprocess images
- models

## Demo

https://github.com/hollance/MobileNet-CoreML

|MobileNet|SqueezeNet|
|--|--|
|<img src="assets/coreml_mobilenet.png" width="400">|<img src="assets/coreml_sqeezenet.png" width="400">|

## TensorFlow

- TensorFlow: https://github.com/samjabrahams/tensorflow-on-raspberry-pi
- TensorFlow Lite: https://medium.com/@haraldfernengel/compiling-tensorflow-lite-for-a-raspberry-pi-786b1b98e646
<img src="assets/tflite-architecture.jpg" width=500>

In [ ]:
# https://github.com/tensorflow/models/tree/master/research/slim/nets/mobilenet

!git clone https://github.com/tensorflo

## Movidius

## Embedded Learning Library (ELL)

|For|Try|
|--|--|
|iOS|CoreML, TensorFlow Lite|
|Android|TensorFlow Lite|
|Raspberry Pi|TensorFlow, TensorFlow Lite|
|Raspberry Pi for Vision|Movidius|
|Smaller than Raspberry Pi|Embedded Learning Library|
|FPGA|??|